#### Task 1: Simple vector embedding generation

**Objective:**
Generate vector embeddings from text data.

**Task Description:**

- load huggingface embedding model (`model_name="sentence-transformers/all-mpnet-base-v2"`)
- embed simple text queries

How to select the right embedding model: [MTEB - Massive Text Embedding Benchmark](https://huggingface.co/blog/mteb)

**Useful links:**

- [Langchain Chroma](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/)


In [1]:
from langchain_huggingface import HuggingFaceEmbeddings

# ADD HERE YOUR CODE
embedding_model = HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2")

/workspaces/TeamNovaBot/.venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/workspaces/TeamNovaBot/.venv/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
text = "This is a test document."

# ADD HERE YOUR CODE
# Perform vector search
query_vector = text

print(f"Embedding vector length: {len(query_vector)}")
print(query_vector[:10])

Embedding vector length: 24
This is a 


#### Task 2: Generate embedding vectors with custom dataset

**Objective:**
Load custom dataset, preprocess it and generate vector embeddings.

**Task Description:**

- load pdf document "AI_Book.pdf" via langchain document loader: `PyPDFLoader`
- use RecursiveCharacterTextSplitter to split documents into chunks
- generate embeddings for single documents

**RecursiveCharacterTextSplitter:**

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough. The default list is `["\n\n", "\n", " ", ""]`. This has the effect of trying to keep all paragraphs (and then sentences, and then words) together as long as possible, as those would generically seem to be the strongest semantically related pieces of text.

**Useful links:**

- [Langchain PyPDFLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.pdf.PyPDFLoader.html)
- [Langchain RecursiveCharacterTextSplitter](https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html)


In [34]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
import re

pdf_doc = "./AI_Book.pdf"

# Create pdf data loader
# ADD HERE YOUR CODE
loader = PyPDFLoader(
    file_path = pdf_doc,
    # passwort = "my-passwort"
    extract_images = False,
)

# Load and split documents in chunks
# ADD HERE YOUR CODE
pages_chunked = RecursiveCharacterTextSplitter().split_documents(documents = loader.load())

# Function to clean text by removing invalid unicode characters, including surrogate pairs
def clean_text(text):
    # Remove surrogate pairs
    text = re.sub(r'[\ud800-\udfff]', '', text)
    # Optionally remove non-ASCII characters (depends on your use case)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    return text

def clean_and_create_document(chunk):
    cleaned_text = clean_text(chunk.page_content)
    return Document(page_content = cleaned_text, metadata = chunk.metadata)

pages_chunked_cleaned1 = [clean_and_create_document(chunk) for chunk in pages_chunked]
pages_chunked_cleaned = [clean_text(chunk.page_content) for chunk in pages_chunked]
print(pages_chunked[0])

page_content='Aurélien GéronHands-on Machine Learning with
Scikit-Learn, Keras, and
TensorFlow
Concepts, Tools, and Techniques to
Build Intelligent SystemsSECOND EDITION
Boston Farnham Sebastopol Tokyo Beijing Boston Farnham Sebastopol Tokyo Beijing' metadata={'source': './AI_Book.pdf', 'page': 2}


In [4]:
print(pages_chunked[1])

page_content='978-1-492-03264-9
[LSI]Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow
by Aurélien Géron
Copyright © 2019 Aurélien Géron. All rights reserved.
Printed in the United States of America.
Published by O’Reilly Media, Inc., 1005 Gravenstein Highway North, Sebastopol, CA 95472.
O’Reilly books may be purchased for educational, business, or sales promotional use. Online editions are
also available for most titles ( http://oreilly.com ). For more information, contact our corporate/institutional
sales department: 800-998-9938 or corporate@oreilly.com .
Editor:  Nicole Tache
Interior Designer:  David FutatoCover Designer:  Karen Montgomery
Illustrator:  Rebecca Demarest
June 2019:  Second Edition
Revision History for the Early Release
2018-11-05: First Release
2019-01-24: Second Release
2019-03-07: Third Release
2019-03-29: Fourth Release
2019-04-22: Fifth Release
See http://oreilly.com/catalog/errata.csp?isbn=9781492032649  for release details.
The O’Reilly logo 

In [5]:
print(f"Number of text chunks: {len(pages_chunked)}")

Number of text chunks: 507


#### Task 3: Store vector embeddings from pdf document to ChromaDB vector database.

**Objective:**

Store vector embeddings into ChromaDB to store knowledge.

**Task Description:**

- create chromadb client
- create chromadb collection
- create langchain chroma db client
- store text document chunks and vector embeddings to vector databases

**Useful links:**

- [Langchain How To](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/#initialization-from-client)


In [37]:
from langchain_chroma import Chroma
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

client = chromadb.HttpClient(
    host     = "localhost",
    port     = 8000,
    ssl      = False,
    headers  = None,
    settings = Settings(allow_reset = True, anonymized_telemetry = False),
    tenant   = DEFAULT_TENANT,
    database = DEFAULT_DATABASE,
)

# ADD HERE YOUR CODE
# Create a collection
collection = client.get_or_create_collection("collection_name")

# ADD HERE YOUR CODE
# Create chromadb
vector_db_from_client = Chroma(
    client = client,
    collection_name = "collection_name",
    embedding_function = embedding_model,
)

In [38]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(pages_chunked[:50]))]
#uuids = [str(uuid4()) for _ in range(len(pages_chunked_cleaned1[:50]))]

# ADD HERE YOUR CODE
vector_db_from_client.add_documents(documents = pages_chunked[:50], id = uuids)
#vector_db_from_client.add_documents(documents = pages_chunked_cleaned1, id = uuids)

['f2402326-2047-466e-acc9-8ac69b511b8f',
 '4f084334-f51a-4f66-bfff-d48171293324',
 'e14de60d-5b2f-4ec1-8c53-b9a9f1ac9159',
 'c082fec5-1fa6-49b6-8480-241f9ae004ee',
 '5a1d587f-020a-4083-af6a-40bfdfc93025',
 '5b8cf9e9-8131-47c8-99bf-4d7cbf2ea976',
 '34b4c49d-b802-4e1a-90d3-f44a323b7e7d',
 '9ef13394-b6ad-4ce3-b88e-d20284003ba4',
 'b00299e8-5b90-4162-8c68-9636fce05256',
 '36ebb99a-2cfa-41b7-97ce-e0b518b0e96b',
 '211bb5f9-37f4-4c9b-a9c5-8b2496f2c8a0',
 '2f97f893-6db4-4237-aee2-c04f7eae2f98',
 '6b718c53-2948-4672-8ccc-d7cb0766337a',
 '5ab563d3-f81d-4c8c-a0c9-098752eadd09',
 'f4595396-7539-4ae3-896e-4c79db8df5f9',
 'dc477949-0515-4684-8e00-713a26d069c2',
 'd842df6f-93c8-4b8e-8a10-615eb572b3b2',
 'd65222ff-7f43-46c9-98c7-c0e98e74fd88',
 '8463f852-4f52-49ca-af4e-19bbcff3238a',
 '2049fdbd-ee2b-4135-aac4-a9c36d8ef2a4',
 'cee1a5cd-a1fd-4a04-bfa8-11ef5896175b',
 'a809a51d-7445-4065-94ab-7538203c9e26',
 'd6ae8d1d-0410-4fd7-aac1-22cb6356316e',
 'b82b1ba4-4f56-44ad-891e-95365170e529',
 'e6be14f1-1117-

In [21]:
client.count_collections()

1

In [22]:
# client.delete_collection("ai_model_book")

#### Task 4: Access ChromaDB and perform vector search

**Objective:**

Use query to perform vector search against ChromaDB vector database

**Task Description:**

- define query
- run vector search
- print k=3 most similar documents

**Useful links:**

- [Langchain Query ChromaDB](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/#query-directly)


In [43]:
search_query = "Types of Machine Learning Systems"

results = vector_db_from_client.similarity_search("search_query", k = 3,)

for res in results:
    print(res.page_content)
    print(res.metadata)
    print("\n")

Exception: {"error":"InvalidArgumentError","message":"Expected where to have exactly one operator, got {}"}